# Artificial and Computational Intelligence: Assignment 2

## Game Playing in Python

1. 2023aa05727&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;[G. Ankur Vatsa](mailto:2023aa05727@wilp.bits-pilani.ac.in)                     % effort contribution
2. 2023aa05716&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;[NIDASANAMETLA SREE SITAMAHALAKSHMI](mailto:2023aa05716@wilp.bits-pilani.ac.in) % effort contribution
3. 2023aa05828&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;[RANDHAWANE SANTOSH](mailto:2023aa05828@wilp.bits-pilani.ac.in)                 % effort contribution
4. 2023aa05256&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;[PRASENJIT SAMANTA](mailto:2023aa05256@wilp.bits-pilani.ac.in)                  % effort contribution
5. 2023aa05348&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;[VEDAGIRI SAI KRISHNA](mailto:2023aa05348@wilp.bits-pilani.ac.in)               % effort contribution

### 1.	Define the environment in the following block

#List the PEAS decription of the problem here in this markdown block
<h2>PEAS Environment</h2>
<dl>
  <dt>Performance</dt>
  <dd>Every player aims to get a higher score than the opponent or secure a tie if winning is not feasible. The efficiency of the Minimax algorithm with Alpha-Beta pruning is crucial for larger game states. It ensures that the algorithm performs well, especially as the size of the number set increases.</dd>
  <dt>Environments</dt>
    <dd>
      <ul>
        <li>The environment consists of the set of numbers {1, 2, 3, ..., n} where n will be provided by the player as an input.</li>
        <li>Two player’s game</li>
        <li>It also includes the rules of the game, such as 
          <ul>
            <li>players taking turns choosing numbers</li>
            <li>numbers being removed from the set after being chosen</li>
            <li>the condition for ending the game</li>
          </ul>  
        </li>
      </ul>
    </dd>
  <dt>Sensors</dt>
  <dd>The sensors provide information about the state of the game, including the current set of available numbers, the sum chosen by the opponent in the previous turn, possibly the choices made by the opponent and player scores</dd>
  <dt>Actuators</dt>
  <dd>
    <ul>
      <li>It helps to prune unnecessary branches and evaluates different move possibilities for each player</li>
      <li>It is using the mechanism of Minimax with alpha-beta pruning for decision making</li>
    </ul>
  </dd>
</dl>

### Catch-Up with Numbers 

### Adding necessary import

In [10]:
import random
from collections import deque
from abc import ABC, abstractmethod

<h2>Class: GameState</h2>
<dl>
  <dd>
      Represents the current state of the game, including available numbers, player scores, etc.
  </dd>
</dl>


In [11]:
class GameState:
    def __init__(self, n):
        self.n = n
        self.available_numbers = set(range(1, n + 1))
        self.current_player_score = 0
        self.other_player_score = 0

    def copy(self):
        copy_state = GameState(self.n)  # Create an empty GameState object
        copy_state.available_numbers = self.available_numbers.copy()  # Assign copied available numbers
        copy_state.current_player_score = self.current_player_score
        copy_state.other_player_score = self.other_player_score
        return copy_state

<h2>Class: GameStrategy (Abstract Class)</h2>
<dl>
  <dd>
      Defines the template for different game strategies. It provides methods for making moves and getting potential numbers to choose from
  </dd>
</dl>


In [12]:
"""
GameStrategy (Abstract Base Class): Defines the structure of a game strategy, 
including the abstract methods move() and get_candidate_numbers(). This class
serves as a blueprint for specific strategies.
"""
class GameStrategy(ABC):
    def __init__(self):
        pass

    @abstractmethod
    def move(self, game_state: GameState):
        pass

    def get_candidate_numbers(self, game_state: GameState):
        available_numbers = game_state.available_numbers
        player_score = game_state.current_player_score  # Adjust for player 2 if needed
        opponent_score = game_state.other_player_score  # Adjust for player 1 if needed

        if player_score == opponent_score:
            # If scores are equal, choose all available numbers
            return list(available_numbers)
        elif player_score > opponent_score:
            # When leading, we prioritize choosing smaller numbers:
            #   - Reduce risk of exceeding opponent's score and losing the lead
            #   - Potentially maintain or increase the lead advantage.
            return sorted(list(available_numbers))
        else:
            # If behind, choose all available numbers that don't exceed the opponent's score
            return list(available_numbers)


<h2>Class: MinimaxStrategy</h2>
<dl>
  <dd>
      Implements the Minimax algorithm with alpha-beta pruning. It determines the best move for the current player by considering the opponent's potential moves and maximizing the player's score while minimizing the opponent's score.
  </dd>
</dl>


In [19]:

"""
MinimaxStrategy class has the combined logic of both Minimax and ⍺-β pruning techniques,
it also incorporates the backward induction approach to find the best move for the player
"""
class MinimaxStrategy(GameStrategy):
    def __init__(self, maximizing_player: bool=True):
        super().__init__()
        self.maximizing_player = maximizing_player

    """
    move() method: applies ⍺-β pruning to efficiently search the game tree and avoid 
    exploring unpromising branches. This method selects the best move for the current
    player based on the Minimax algorithm, considering the opponent's potential moves 
    and maximizing the player's score while minimizing the opponent's score.
    """
    def move(self, game_state: GameState):
        if not game_state.available_numbers:
            print("Error: No available numbers")
            return None

        best_move = None
        alpha = float("-inf")
        beta = float("inf")

        queue = deque([game_state.copy()])

        while queue:
            current_state = queue.popleft()

            for number in current_state.available_numbers:
                new_state = current_state.copy()
                new_state.available_numbers.remove(number)
                new_state_score = new_state.current_player_score + number

                # Check if the game is over for the opponent
                if not new_state.available_numbers:
                    score = new_state_score if self.maximizing_player else 0
                else:
                    # Simulate opponent's move
                    opponent_score = self._simulate_opponent_move(new_state.available_numbers)
                    new_state.other_player_score = opponent_score

                    # Recursively evaluate the next move
                    next_move_score = self._minimax(new_state, alpha, beta, maximizing=not self.maximizing_player)
                    score = new_state_score - opponent_score if self.maximizing_player else next_move_score

                    score, _ = self._alpha_beta_pruning(alpha, beta, score, number)

                # Check if pruning condition is met and break out of the loop if necessary
                if self.maximizing_player:
                    alpha = max(alpha, score)
                    if alpha >= beta:
                        break
                else:
                    beta = min(beta, score)
                    if beta <= alpha:
                        break

                if score > alpha:
                    alpha = score
                    best_move = number

                queue.append(new_state)

        return best_move

    def _alpha_beta_pruning(self, alpha, beta, score, number):
            if self.maximizing_player:
                alpha = max(alpha, score)
                if alpha >= beta:
                    return alpha, None
            else:
                beta = min(beta, score)
                if beta <= alpha:
                    return beta, None
            return score, number

    """
    _minimax() is a private method that performs the recursive evaluation of 
    game states, considering moves of both maximizing and minimizing players
    """
    def _minimax(self, game_state, alpha, beta, maximizing=True):
        if not game_state.available_numbers:
            return game_state.current_player_score if maximizing else 0

        best_score = float("-inf") if maximizing else float("inf")
        for number in game_state.available_numbers:
            new_state = game_state.copy()
            new_state.available_numbers.remove(number)
            new_state_score = new_state.current_player_score + number

            # Simulate opponent's move
            opponent_score = self._simulate_opponent_move(new_state.available_numbers)
            new_state.other_player_score = opponent_score

            score = new_state_score - opponent_score + self._minimax(new_state, alpha, beta, maximizing=not maximizing)
            best_score = max(best_score, score) if maximizing else min(best_score, score)
            alpha = max(alpha, score) if maximizing else min(alpha, score)
            if alpha >= beta:
                break
            if not maximizing and beta <= alpha:
                break
        return best_score

    def _simulate_opponent_move(self, available_numbers):
        if not available_numbers:
            return 0  # Return a default score when no numbers are available
        return min(available_numbers)


<h2>Class: Player</h2>
<dl>
  <dd>
      Represents a player in the game, including their name, score, and chosen strategy.
  </dd>
</dl>


In [20]:
class Player:
    def __init__(self, name: str):
        self.name = name
        self.score = 0
        self.strategy = None

    def choose_strategy(self, c: int = None) -> bool:
        if c is None:
            choice = int(input(f"{self.name}, select (1) to become maximizer or (2) to be minimizer? "))
        else:
            choice = c
    
        if choice not in [1, 2]:
            print(f"Invalid choice. Defaulting {self.name} to maximizer.")
            choice = 1
        
        self.strategy = MinimaxStrategy(True) if choice == 1 else MinimaxStrategy(False)

        return not choice
    
    def get_strategy_name(self) -> str:
        if self.strategy:
            return "Maximizer" if isinstance(self.strategy, MinimaxStrategy) and self.strategy.maximizing_player else "Minimizer"
        else:
            return "No strategy chosen"

    def make_move(self, game_state: GameState):
        try:
            move = self.strategy.move(game_state.copy())
            if move is not None:
                print(f"{self.name} chooses: {move}")
            else:
                print(f"{self.name} cannot make a move.")
            return move
        except (IndexError, ValueError) as e:
            print(f"Error making move: {e}")
            return None


<h2>Class: CatchUpGame</h2>
<dl>
  <dd>
      Manages the overall game flow, including determining the starting player, checking the game state, identifying the winner, and handling turns for each player.
  </dd>
</dl>


In [21]:

class CatchUpGame:
    def __init__(self, n: int, player1: Player, player2: Player):
        self.game_state = GameState(n)
      
        # Randomly choose the starting player and set their preferred strategy
        self.current_player = random.choice([player1, player2])
        print(f"Randomly chose {self.current_player.name} as starting player")
        
        
        # Choose strategy for the current player
        self.current_player.choose_strategy()
        
        self.other_player = player2 if self.current_player == player1 else player1
        c = 2 if self.current_player.strategy == 1 else 1
        self.other_player.choose_strategy(c=c)

    def is_game_over(self):
        return (not self.game_state.available_numbers) or \
           (self.current_player.strategy.move(self.game_state.copy()) is None and 
            self.other_player.strategy.move(self.game_state.copy()) is None)

    def get_winner(self):
        if self.current_player.score > self.other_player.score:
            return self.current_player
        elif self.other_player.score > self.current_player.score:
            return self.other_player
        else:
            return None
    
    def change_PlayerTurn(self):
        if(self.current_player.name == "Player 1"):
            self.current_player.name = "Player 2"
            self.name = "Player 2"
        else:
            self.current_player.name = "Player 1"
            self.name = "Player 1"

    def play(self):
        print(f"Starting game Catch-up Numbers: {self.current_player.name} vs. {self.other_player.name}")

        while not self.is_game_over():
            print(f"Available numbers: {sorted(list(self.game_state.available_numbers))}")
            self.change_PlayerTurn()
            # Player 1's turn
            p1_choice = self.current_player.make_move(self.game_state.copy())
            if p1_choice is not None:
                self.game_state.available_numbers.remove(p1_choice)
                self.current_player.score += p1_choice
                print(f"{self.current_player.name} chooses: {p1_choice}")
            else:
                print(f"{self.current_player.name} cannot make a move.")
                break

            # Player 2's turn
            p2_choice = self.other_player.make_move(self.game_state.copy())
            if p2_choice is not None:
                self.game_state.available_numbers.remove(p2_choice)
                self.other_player.score += p2_choice
                print(f"{self.other_player.name} chooses: {p2_choice}")
            else:
                print(f"{self.other_player.name} cannot make a move.")
                break

            print(f"{self.current_player.name} score: {self.current_player.score}, {self.other_player.name} score: {self.other_player.score}\n")

        winner = self.get_winner()
        if winner:
            print(f"{winner.name} wins!")
        else:
            print("It's a tie!")



<h2>Main function</h2>
<dl>
  <dd>
      Initiate game execution
  </dd>
</dl>


In [22]:
def main():
    player1 = Player("Player 1")
    player2 = Player("Player 2")
    isValidRange = True
    while(isValidRange):
        n = int(input("Enter the range of numbers (n) greater than 1: "))
        if(n>1):
            isValidRange = False
    
    catch_up_game = CatchUpGame(n, player1, player2)
    catch_up_game.play()

In [23]:
if __name__ == "__main__":
    main()

Enter the range of numbers (n) greater than 1: 6
Randomly chose Player 2 as starting player
Player 2, select (1) to become maximizer or (2) to be minimizer? 2
Starting game Catch-up Numbers: Player 2 vs. Player 1
Available numbers: [1, 2, 3, 4, 5, 6]
Player 1 chooses: 1
Player 1 chooses: 1
Player 1 cannot make a move.
Player 1 cannot make a move.
Player 1 wins!
